<a href="https://colab.research.google.com/github/MennaFawzy/Pyspark/blob/main/Practical_Day_1_DF_SQL_Students_DM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Labs 1 and 2 PySpark:**

In these labs we will be using the "[[NeurIPS 2020] Data Science for COVID-19 (DS4C)](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset?select=PatientInfo.csv)" dataset, retrieved from [Kaggle](https://www.kaggle.com/) on 1/6/2022, for educational non commercial purpose, License
[CC BY-NC-SA 4.0
](https://creativecommons.org/licenses/by-nc-sa/4.0/)


The csv file that we will be using in this lab is **PatientInfo**.

## PatientInfo.csv

**patient_id**
the ID of the patient

**sex**
the sex of the patient

**age**
the age of the patient

**country**
the country of the patient

**province**
the province of the patient

**city**
the city of the patient

**infection_case**
the case of infection

**infected_by**
the ID of who infected the patient


**contact_number**
the number of contacts with people

**symptom_onset_date**
the date of symptom onset

**confirmed_date**
the date of being confirmed

**released_date**
the date of being released

**deceased_date**
the date of being deceased

**state**
isolated / released / deceased

### Import the pyspark and check it's version

In [1]:
!pip install pyspark

### Import and create SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Patientinfo").getOrCreate()

### Load the PatientInfo.csv file and show the first 5 rows

In [3]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [4]:
spark.read.csv('/content/PatientInfo.csv',header=True,inferSchema=True)

DataFrame[patient_id: bigint, sex: string, age: string, country: string, province: string, city: string, infection_case: string, infected_by: string, contact_number: string, symptom_onset_date: string, confirmed_date: date, released_date: date, deceased_date: date, state: string]

In [5]:
df=spark.read.csv('/content/PatientInfo.csv',header=True,inferSchema=True)

In [6]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|         NULL|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|         NULL|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              NULL|    2020-01-30|   202

### Display the schema of the dataset

In [7]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)



### Display the statistical summary

In [8]:
df.summary().show()

+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|      contact_number|symptom_onset_date|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1346|                 791|               690|    5165|
|   mean|2.8636345618679576E9|  NULL|NULL|      NULL|    NULL|          NULL|                NULL|2.2845944015643125E9|1.6772572523506988E7|              NULL|    NULL|
| stddev| 2.074210725277473E9|  NULL|NULL|      NULL|    NULL|          NULL|                NULL|1.5265072953383324E9| 3.093097580985502E8|              N

### Using the state column.
### How many people survived (released), and how many didn't survive (isolated/deceased)?

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
state_counts = df.groupBy("state").agg(count("*").alias("count"))

# Show the survival counts
state_counts.show()

+--------+-----+
|   state|count|
+--------+-----+
|isolated| 2158|
|released| 2929|
|deceased|   78|
+--------+-----+



In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, sum
count_df = df.withColumn("survived",when(col("state") == "released", 1).otherwise(0)).withColumn(
    "didn't survive",when((col("state") == "isolated") | (col("state") == "deceased"), 1).otherwise(0))

result = count_df.agg(sum("survived").alias("survived"), sum("didn't survive").alias("didn't survive"))
result.show()

+--------+--------------+
|survived|didn't survive|
+--------+--------------+
|    2929|          2236|
+--------+--------------+



### Display the number of null values in each column

In [11]:
null_counts = df.select([sum(col(col_name).isNull().cast("int")).alias(col_name) for col_name in df.columns])
null_counts.show()


+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id| sex| age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|1122|1380|      0|       0|  94|           919|       3819|          4374|              4475|             3|         3578|         5099|    0|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



## Data preprocessing

### Fill the nulls in the deceased_date with the released_date.
- You can use <b>coalesce</b> function

In [12]:
from pyspark.sql.functions import coalesce
df = df.withColumn("deceased_date", coalesce(df["deceased_date"], df["released_date"]))
df.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              NULL|    2020-01-30|

### Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows. Print the schema.
- <b> Hint: You need to typecast these columns as date first <b>

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, to_date
from pyspark.sql.types import IntegerType
df = df.withColumn("confirmed_date", to_date(col("confirmed_date"), "yyyy-MM-dd"))
df = df.withColumn("deceased_date", to_date(col("deceased_date"), "yyyy-MM-dd"))
df = df.withColumn("no_days", datediff(col("deceased_date"), col("confirmed_date")).cast(IntegerType()))
df.show(5)
df.printSchema()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|

### Add a is_male column if male then it should yield true, else then False

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
df = df.dropna(subset=["sex"])
df = df.withColumn("sex", when(df["sex"] == "male", True).otherwise(False))
df.show()

+----------+-----+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|patient_id|  sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|
+----------+-----+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+
|1000000001| true|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|
|1000000002| true|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|
|1000000003| true|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|

### Add a is_dead column if patient state is not released then it should yield true, else then False

- Use <b>UDF</b> to perform this task. user define function
- However, UDF is not recommended there is no built in function can do the required operation.
- UDF is slower than built in functions.

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf
def is_dead(state):
    return state != "released"
is_dead_udf = udf(is_dead, BooleanType())
df = df.withColumn("is_dead", is_dead_udf(col("state")))
df.show()


+----------+-----+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+
|patient_id|  sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_dead|
+----------+-----+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+
|1000000001| true|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|  false|
|1000000002| true|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|  false|
|1000000003| true|50s|  Korea|   Seoul|   Jongno-gu|contact 

### Change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20

In [17]:
from pyspark.sql.functions import floor, col, regexp_replace
df = df.withColumn("age", regexp_replace(col("age"), "s", "").cast("integer"))
df = df.withColumn("age_bin", (floor(col("age") / 10) * 10).cast("integer"))


In [18]:
df.show()

+----------+-----+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|  sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_dead|age_bin|
+----------+-----+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001| true| 50|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|  false|     50|
|1000000002| true| 30|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|  false|     30|
|1000000003| true| 5

### Change age, and no_days  to be typecasted as Double

In [19]:
from pyspark.sql.functions import floor, col, regexp_replace
from pyspark.sql.types import DoubleType
df = df.withColumn("no_days", col("no_days").cast(DoubleType()))

In [20]:
df.show()

+----------+-----+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|  sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_dead|age_bin|
+----------+-----+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001| true| 50|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|   13.0|  false|     50|
|1000000002| true| 30|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|   2020-03-02|released|   32.0|  false|     30|
|1000000003| true| 5

### Drop the columns
["patient_id","sex","infected_by","contact_number","released_date","state",
"symptom_onset_date","confirmed_date","deceased_date","country","no_days",
"city","infection_case"]

In [21]:
columns_to_drop = ["patient_id", "sex", "infected_by", "contact_number", "released_date","state", "symptom_onset_date", "confirmed_date", "deceased_date", "country",
"no_days", "city", "infection_case"]
df = df.drop(*columns_to_drop)
df.show()

+---+--------+-------+-------+
|age|province|is_dead|age_bin|
+---+--------+-------+-------+
| 50|   Seoul|  false|     50|
| 30|   Seoul|  false|     30|
| 50|   Seoul|  false|     50|
| 20|   Seoul|  false|     20|
| 20|   Seoul|  false|     20|
| 50|   Seoul|  false|     50|
| 20|   Seoul|  false|     20|
| 20|   Seoul|  false|     20|
| 30|   Seoul|  false|     30|
| 60|   Seoul|  false|     60|
| 50|   Seoul|  false|     50|
| 20|   Seoul|  false|     20|
| 80|   Seoul|   true|     80|
| 60|   Seoul|  false|     60|
| 70|   Seoul|  false|     70|
| 70|   Seoul|  false|     70|
| 70|   Seoul|  false|     70|
| 20|   Seoul|  false|     20|
| 70|   Seoul|  false|     70|
| 70|   Seoul|  false|     70|
+---+--------+-------+-------+
only showing top 20 rows



### Recount the number of nulls now

> Indented block



In [22]:
null_counts = df.select([sum(col(col_name).isNull().cast("int")).alias(col_name) for col_name in df.columns])
# Show the null value counts
null_counts.show()

+---+--------+-------+-------+
|age|province|is_dead|age_bin|
+---+--------+-------+-------+
|261|       0|      0|    261|
+---+--------+-------+-------+



## Now do the same but using SQL select statement

### From the original Patient DataFrame, Create a temporary view (table).

In [23]:
df = spark.read.csv("PatientInfo.csv", header=True, inferSchema=True)
df.createOrReplaceTempView("patient_view")

### Use SELECT statement to select all columns from the dataframe and show the

1.   List item
2.   List item

output.

In [24]:
result = spark.sql("SELECT * FROM patient_view")
result.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|         NULL|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|         NULL|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              NULL|    2020-01-30|

### *Using SQL commands*, limit the output to only 5 rows

In [25]:
result = spark.sql("SELECT * FROM patient_view LIMIT 5")
result.show()


+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       NULL|            75|        2020-01-22|    2020-01-23|   2020-02-05|         NULL|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       NULL|            31|              NULL|    2020-01-30|   2020-03-02|         NULL|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              NULL|    2020-01-30|   202

### Select the count of males and females in the dataset

---



In [26]:
result = spark.sql("select sex, count(*) as count from patient_view group by sex")
result.show()

+------+-----+
|   sex|count|
+------+-----+
|  NULL| 1122|
|female| 2218|
|  male| 1825|
+------+-----+



In [27]:
result = spark.sql("select sex, count(*) as count from patient_view where sex is not null group by sex")
result.show()

+------+-----+
|   sex|count|
+------+-----+
|female| 2218|
|  male| 1825|
+------+-----+



\### How many people did survive, and how many didn't?

In [28]:
survived_count = df.filter(df["state"] == "released").count()
not_survived_count = df.filter(df["state"] != "released").count()
print("Number of people who survived (released):", survived_count)
print("Number of people who didn't survive (isolated/deceased):", not_survived_count)


Number of people who survived (released): 2929
Number of people who didn't survive (isolated/deceased): 2236


### Now, let's perform some preprocessing using SQL:
1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [29]:
preprocessed_df = spark.sql("""SELECT sex,CAST(SUBSTRING(age, 1, LENGTH(age) - 1) AS DOUBLE) AS age,province,state FROM patient_view""")
preprocessed_df.show()

+------+----+--------+--------+
|   sex| age|province|   state|
+------+----+--------+--------+
|  male|50.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|  male|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|20.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|female|60.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|80.0|   Seoul|deceased|
|female|60.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|70.0|   Seoul|released|
|female|70.0|   Seoul|released|
+------+----+--------+--------+
only showing top 20 rows

